In [1]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, html, datetime
import openpyxl
from openpyxl import Workbook 
from os import listdir
from os.path import isfile, join
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator   
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')

In [15]:
pl=sql_todf("""
SELECT UPPER(LTRIM(RTRIM(DisplayValue))) [Selections], PK_PickList [ML_Value], PK_PickListType [PLT]
FROM vwPickLists  
WHERE PK_PickListType > 410
ORDER BY PK_PickListType DESC
""", connstr)  
s=list(pl.loc[pl['PLT']==445]['Selections'])
pl['Selections'] = pl['Selections'].apply(lambda x: re.sub('[^A-Za-z0-9\s]','',str(x)))
print(s)
pl.loc[pl['PLT']==445]

["TRADITIONAL SECURITY CONTROLS (E.G., STATIC ACCESS CONTROL LISTS, REQUESTOR'S IP ADDRESS)", "LEAST-PRIVILEGE CONTROLS (E.G., REQUESTOR'S IDENTITY ATTRIBUTES, DEVICE RISK, ACCESS REQUEST DATE/TIME)", 'DYNAMIC CONTROLS (E.G., JUST-IN-TIME AND JUST-ENOUGH ACCESS PRINCIPLES, CONTINUOUS RISK-BASED DETERMINATIONS)', 'UNKNOWN']


,Selections,ML_Value,PLT
69,TRADITIONAL SECURITY CONTROLS EG STATIC ACCESS...,4830,445
70,LEASTPRIVILEGE CONTROLS EG REQUESTORS IDENTITY...,4831,445
71,DYNAMIC CONTROLS EG JUSTINTIME AND JUSTENOUGH ...,4832,445
72,UNKNOWN,4833,445


In [17]:
filepath = r'C:\Users\timko\Downloads\Questionnaire\HVA Data Protection Questionnaire v0.1.xlsx'
path=r'C:\Users\timko\Downloads\Questionnaire\\'
files = [f for f in listdir(path) if re.search('^HVA.*1\.xlsx', f)]
#print(files)
xlfile=files[2]
#xlsx= openpyxl.load_workbook(f'{path}{xlfile}')
xlfile 
 

'HVA Data Protection Questionnaire v0.1.xlsx'

In [18]:
converts={
    'Selections':lambda x: re.sub('\d{,2}\.','',str(x)).upper().strip()
    , 'ML':lambda x: str(x).strip()
    , 'SML':lambda x: re.sub('[^A-Z0-9]','',str(x)).strip()  }
df=pd.read_excel(f'{path}{xlfile}', converters=converts,  sheet_name=4, header=2, usecols='B:I').reset_index()
df = df.rename(columns={'ML Score': 'ML_Score','ML Scored': 'ML_Scored' })
for c in ['ID','Question','SML']:
    df[c] = df[c].replace(to_replace=[0, '0'], method='ffill')
df["ID"].fillna(method='ffill', inplace=True) 
df.drop(columns=['Check Answer', 'index'], inplace=True)
df.loc[df['ID']=='DP17']
 


,ID,Question,Selections,SML,ML,ML_Score,ML_Scored
167,DP17,Which of the following inputs are used to make...,"TRADITIONAL SECURITY CONTROLS (EG, STATIC ACCE...",SML1,1,1,0
168,DP17,Which of the following inputs are used to make...,"LEAST-PRIVILEGE CONTROLS (EG, REQUESTOR'S IDEN...",SML2,2,1,0
169,DP17,Which of the following inputs are used to make...,"DYNAMIC CONTROLS (EG, JUST-IN-TIME AND JUST-EN...",SML3,3,1,0
170,DP17,Which of the following inputs are used to make...,UNKNOWN,SML3,1,0.001,0


In [19]:
df = df.loc[ df["ID"].str.contains('^\w\w\d(\w){0,5}$', na=False, regex=True) ]  
df = df.loc[ df["ML"].str.contains('\d', na=False,  regex=True, case=False) ]  
df['ML_Score'] = df['ML_Score'].apply(lambda s: re.sub('[^0-9\.]','',str(s)))
df['Selections'] = df['Selections'].apply(lambda x: re.sub('[^A-Za-z0-9\s]','',str(x)))
df.loc[df['ID']=='DP17']

,ID,Question,Selections,SML,ML,ML_Score,ML_Scored
167,DP17,Which of the following inputs are used to make...,TRADITIONAL SECURITY CONTROLS EG STATIC ACCESS...,SML1,1,1,0
168,DP17,Which of the following inputs are used to make...,LEASTPRIVILEGE CONTROLS EG REQUESTORS IDENTITY...,SML2,2,1,0
169,DP17,Which of the following inputs are used to make...,DYNAMIC CONTROLS EG JUSTINTIME AND JUSTENOUGH ...,SML3,3,1,0
170,DP17,Which of the following inputs are used to make...,UNKNOWN,SML3,1,0.001,0


In [20]:
dfp=df.groupby(['ID'], as_index=False).agg({'Selections':list}) 
pr = picklist_recommender(ctx.config['connstr'], reset_cache=True, picklist_where=' PK_PickListType > 410 ') 
dfp['PLT']=dfp['Selections'].apply(lambda s: pr.recommend(s)['PK_PicklistType'])
pr.to_cache() 

dff=pd.merge(df,dfp.loc[:,['ID','PLT']], how='left', left_on='ID', right_on='ID')
dff=pd.merge(dff,pl, how='left', left_on=['Selections','PLT'], right_on=['Selections','PLT'])
dff.ML_Value=dff.ML_Value.fillna(0).astype('int32').astype('str')
dff.loc[dff['ID']=='DP17']

,ID,Question,Selections,SML,ML,ML_Score,ML_Scored,PLT,ML_Value
136,DP17,Which of the following inputs are used to make...,TRADITIONAL SECURITY CONTROLS EG STATIC ACCESS...,SML1,1,1,0,445,4830
137,DP17,Which of the following inputs are used to make...,LEASTPRIVILEGE CONTROLS EG REQUESTORS IDENTITY...,SML2,2,1,0,445,4831
138,DP17,Which of the following inputs are used to make...,DYNAMIC CONTROLS EG JUSTINTIME AND JUSTENOUGH ...,SML3,3,1,0,445,4832
139,DP17,Which of the following inputs are used to make...,UNKNOWN,SML3,1,0.001,0,445,4833


In [21]:
df1=dff
df1.SML.fillna(method='ffill', inplace=True)
df1=df1.loc[df1['ML_Score'] != '0']
df1['sortorder'] = range(1, len(df1)+1)
#df1['ML_Score']=df1['ML_Score'].astype(float)
sql=SQL_INSERT_FROM_DF(df1.loc[:, ['ID', 'SML', 'ML','ML_Score', 'ML_Scored', 'ML_Value', 'sortorder']], TARGET='@SASCORING')
with open(f'{ctx.get_dest()}script.sql', 'w') as f:
    f.write('\n'.join(sql))
df1.loc[df1['ID']=='DP17']

--DECLARE @T AS TABLE (ID NVARCHAR(MAX), SML NVARCHAR(MAX), ML NVARCHAR(MAX), ML_Score NVARCHAR(MAX), ML_Scored NVARCHAR(MAX), ML_Value NVARCHAR(MAX), sortorder INT)


,ID,Question,Selections,SML,ML,ML_Score,ML_Scored,PLT,ML_Value,sortorder
136,DP17,Which of the following inputs are used to make...,TRADITIONAL SECURITY CONTROLS EG STATIC ACCESS...,SML1,1,1,0,445,4830,95
137,DP17,Which of the following inputs are used to make...,LEASTPRIVILEGE CONTROLS EG REQUESTORS IDENTITY...,SML2,2,1,0,445,4831,96
138,DP17,Which of the following inputs are used to make...,DYNAMIC CONTROLS EG JUSTINTIME AND JUSTENOUGH ...,SML3,3,1,0,445,4832,97
139,DP17,Which of the following inputs are used to make...,UNKNOWN,SML3,1,0.001,0,445,4833,98
